In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
from dateutil.parser import parse
import numpy as np

import warnings
import datetime as dt
warnings.filterwarnings('ignore')

from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()

In [2]:
events = pd.read_csv('data/events_up_to_01062018.csv', low_memory = False)
events.person.describe()

count      2341681
unique       38829
top       c76b8417
freq          4438
Name: person, dtype: object

In [3]:
prediction_set = pd.read_csv('data/trocafone_kaggle_test.csv', low_memory = False)
training_set = pd.read_csv('data/labels_training_set.csv', low_memory=False)

print('PREDICCION ' , prediction_set.person.describe())
print('ENTRENAMIENTO ', training_set.person.describe())
print('ENTRENAMIENTO 1s ', training_set[training_set['label'] == 1].shape)
print('ENTRENAMIENTO 0s ', training_set[training_set['label'] == 0].shape)

frecuencia = events[['person']]
frecuencia['cant_eventos'] = 1
frecuencia = pd.DataFrame({'cant_eventos': frecuencia.groupby('person').aggregate(sum)['cant_eventos']}).reset_index()
events = events.merge(frecuencia, on = 'person', how = 'inner')

prediction_set = prediction_set.merge(events, on = 'person', how = 'inner')
prediction_set = prediction_set[(prediction_set['event'] == 'visited site') | (prediction_set['event'] == 'viewed product') | (prediction_set['event'] == 'conversion') | (prediction_set['event'] == 'checkout') | (prediction_set['event'] == 'brand listing') | (prediction_set['event'] == 'generic listing') | (prediction_set['event'] == 'lead') | (prediction_set['event'] == 'staticpage') ]                                                                       

training_set = events.merge(training_set, on = 'person', how = 'inner')
# training_set= training_set[(training_set['label'] == 1) | ((training_set['cant_eventos'] <= 100) & (training_set['label'] == 0))]


PREDICCION  count        19415
unique       19415
top       8fb666db
freq             1
Name: person, dtype: object
ENTRENAMIENTO  count        19414
unique       19414
top       f7e0b202
freq             1
Name: person, dtype: object
ENTRENAMIENTO 1s  (980, 2)
ENTRENAMIENTO 0s  (18434, 2)


In [4]:
# del info
# del data
del events
del frecuencia

In [5]:
prediction_set.person.describe()

count       953099
unique       19415
top       5059f7fd
freq          2932
Name: person, dtype: object

In [6]:
training_set.person.describe()

count      1171886
unique       19414
top       ffee0f18
freq          3458
Name: person, dtype: object

In [7]:
#ELEGIR SET

In [8]:
#PREDICCION
sets = prediction_set
cols_drop=['url','skus', 'city','search_engine','campaign_source','channel']
cols = ['event','fecha','person', 'sku', 'model', 'condition', 'storage', 'color','cant_eventos']
cols_visit = ['event','fecha','person','new_vs_returning','region','country','device_type','screen_resolution','operating_system_version','browser_version','cant_eventos']
cols_x = ['fecha_x','person','sku_x', 'model_x', 'condition_x', 'storage_x', 'color_x','cant_eventos_x','compras_prod_x','vistas_prod_x', 'otros_x']
rename = {'fecha_x': 'fecha', 'model_x': 'model','sku_x':'sku', 'condition_x': 'condition', 'storage_x': 'storage', 'color_x': 'color','cant_eventos_x': 'cant_eventos', 'compras_prod_x': 'compras_prod', 'vistas_prod_x': 'vistas_prod' ,'otros_x': 'otros'}

In [7]:
#ENTRENAMIENTO
sets = training_set
cols_drop=['url','skus', 'city','search_engine','campaign_source','channel']
cols = ['event','fecha','person', 'sku', 'model', 'condition', 'storage', 'color' ,'cant_eventos', 'label']
cols_visit = ['event','fecha','person','new_vs_returning','region','country','device_type','screen_resolution','operating_system_version','browser_version','cant_eventos', 'label']
cols_x = ['fecha_x','person','sku_x', 'model_x', 'condition_x', 'storage_x', 'color_x','cant_eventos_x','compras_prod_x','vistas_prod_x', 'otros_x', 'label_x']
rename = {'fecha_x': 'fecha', 'model_x': 'model','sku_x':'sku', 'condition_x': 'condition', 'storage_x': 'storage', 'color_x': 'color', 'cant_eventos_x': 'cant_eventos','compras_prod_x': 'compras_prod', 'vistas_prod_x': 'vistas_prod' ,'otros_x': 'otros', 'label_x': 'label'}

In [8]:
del prediction_set
del training_set

# Set Datos

# #Analisis TP1

In [9]:
sets.columns

Index(['timestamp', 'event', 'person', 'url', 'sku', 'model', 'condition',
       'storage', 'color', 'skus', 'search_term', 'staticpage',
       'campaign_source', 'search_engine', 'channel', 'new_vs_returning',
       'city', 'region', 'country', 'device_type', 'screen_resolution',
       'operating_system_version', 'browser_version', 'cant_eventos', 'label'],
      dtype='object')

In [10]:
sets.drop(cols_drop, inplace=True, axis=1)

In [11]:
#Cambiamos el tipo de dato de fecha de timestamp a datetime para un mejor analisis y renombramos a "fecha"
sets[['timestamp']] = sets[['timestamp']].apply(pd.to_datetime)
sets.rename({
    'timestamp' : 'fecha',
}, axis=1, inplace=True)
sets.head()

,fecha,event,person,sku,model,condition,storage,color,search_term,staticpage,new_vs_returning,region,country,device_type,screen_resolution,operating_system_version,browser_version,cant_eventos,label
0,2018-05-18 00:11:27,viewed product,ad93850f,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65,0
1,2018-05-18 00:23:33,viewed product,ad93850f,318.0,iPhone 5s,Muito Bom,64GB,Prateado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65,0
2,2018-05-18 00:16:10,viewed product,ad93850f,5907.0,iPhone 5s,Bom - Sem Touch ID,16GB,Cinza espacial,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65,0
3,2018-05-18 00:14:55,viewed product,ad93850f,6023.0,iPhone 5s,Bom - Sem Touch ID,16GB,Dourado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65,0
4,2018-05-18 00:11:26,ad campaign hit,ad93850f,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65,0


vamos a crear las columnas compras_prod, vistas_prod, otros para productos y visitas_sitio para persona, de la forma de One Hot Encoder

In [12]:
#Detalles de Productos que fueron comprados
productos = sets.loc[:, cols]
productos_comprados = productos[(productos['event'] == 'conversion') | (productos['event'] == 'checkout')]
productos_comprados.drop(['event'], inplace=True, axis=1)
productos_comprados['compras_prod'] = 1
productos_comprados['vistas_prod'] = 0
productos_comprados['otros'] = 0
frecuencia_compras = productos_comprados[['person']]
frecuencia_compras['cant_compras'] = 1
frecuencia_compras = pd.DataFrame({'cant_compras': frecuencia_compras.groupby('person').aggregate(sum)['cant_compras']}).reset_index()
productos_comprados.head()

,fecha,person,sku,model,condition,storage,color,cant_eventos,label,compras_prod,vistas_prod,otros
36,2018-05-14 23:54:19,ad93850f,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,65,0,1,0,0
72,2018-05-18 00:44:49,1b9f7cf6,2834.0,iPhone 6,Bom,64GB,Dourado,25,0,1,0,0
125,2018-05-18 01:00:16,de8fe91b,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,53,0,1,0,0
152,2018-05-18 00:48:20,45baf068,3371.0,Samsung Galaxy S6 Flat,Bom,32GB,Dourado,20,0,1,0,0
211,2018-05-09 00:13:59,99abca5a,10380.0,Motorola Moto G4 Plus,Excelente,32GB,Bambu,819,0,1,0,0


In [13]:
productos_comprados.count()

fecha           36103
person          36103
sku             36103
model           36103
condition       36103
storage         36103
color           36103
cant_eventos    36103
label           36103
compras_prod    36103
vistas_prod     36103
otros           36103
dtype: int64

In [14]:
frecuencia_compras.head()

,person,cant_compras
0,0008ed71,3
1,000c79fe,1
2,001802e4,1
3,0019e639,17
4,001b0bf9,3


In [15]:
print('1s ', productos_comprados[productos_comprados['label'] == 1].shape)
print('0s ', productos_comprados[productos_comprados['label'] == 0].shape)

1s  (2233, 12)
0s  (33870, 12)


In [16]:
# productos_comprados = productos_comprados.drop_duplicates()
# productos_comprados.count()

In [17]:
#Detalles de Productos que fueron visitados
productos_visitados = productos.loc[productos.event == 'viewed product']
productos_visitados.drop(['event'], inplace=True, axis=1)
productos_visitados['compras_prod'] = 0
productos_visitados['vistas_prod'] = 1
productos_visitados['otros'] = 0
frecuencia_vistas = productos_visitados[['person']]
frecuencia_vistas['cant_vistas'] = 1
frecuencia_vistas = pd.DataFrame({'cant_vistas': frecuencia_vistas.groupby('person').aggregate(sum)['cant_vistas']}).reset_index()
productos_visitados.head()

,fecha,person,sku,model,condition,storage,color,cant_eventos,label,compras_prod,vistas_prod,otros
0,2018-05-18 00:11:27,ad93850f,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,65,0,0,1,0
1,2018-05-18 00:23:33,ad93850f,318.0,iPhone 5s,Muito Bom,64GB,Prateado,65,0,0,1,0
2,2018-05-18 00:16:10,ad93850f,5907.0,iPhone 5s,Bom - Sem Touch ID,16GB,Cinza espacial,65,0,0,1,0
3,2018-05-18 00:14:55,ad93850f,6023.0,iPhone 5s,Bom - Sem Touch ID,16GB,Dourado,65,0,0,1,0
5,2018-05-16 02:48:16,ad93850f,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,65,0,0,1,0


In [18]:
productos_visitados.count()

fecha           629233
person          629233
sku             629233
model           629233
condition       629233
storage         629233
color           629233
cant_eventos    629233
label           629233
compras_prod    629233
vistas_prod     629233
otros           629233
dtype: int64

In [19]:
frecuencia_vistas.head()

,person,cant_vistas
0,000c79fe,3
1,001802e4,4
2,0019e639,189
3,001b0bf9,2
4,001ca5ee,52


In [20]:
print('1s ', productos_visitados[productos_visitados['label'] == 1].shape)
print('0s ', productos_visitados[productos_visitados['label'] == 0].shape)

1s  (53773, 12)
0s  (575460, 12)


In [21]:
# productos_visitados = productos_comprados.drop_duplicates()
# productos_visitados.count()

In [22]:
# Eliminamos de productos visitados las personas que estan en productos comprados
productos_visitados = productos_visitados.merge(productos_comprados, on='person', how='left')
productos_visitados = productos_visitados[productos_visitados['fecha_y'].isnull()]
productos_visitados = productos_visitados.loc[:,cols_x]
productos_visitados.rename(columns=rename, inplace=True)
productos_visitados.head()

,fecha,person,sku,model,condition,storage,color,cant_eventos,compras_prod,vistas_prod,otros,label
25576,2018-05-18 00:18:24,9bb3af27,3371.0,Samsung Galaxy S6 Flat,Bom,32GB,Dourado,59,0,1,0,1
25577,2018-05-18 00:15:33,9bb3af27,2741.0,Samsung Galaxy S5,Bom,16GB,Preto,59,0,1,0,1
25578,2018-05-18 00:20:10,9bb3af27,2718.0,Samsung Galaxy S5,Bom,16GB,Branco,59,0,1,0,1
25579,2018-05-18 00:21:58,9bb3af27,3371.0,Samsung Galaxy S6 Flat,Bom,32GB,Dourado,59,0,1,0,1
25580,2018-05-18 00:14:35,9bb3af27,2787.0,Samsung Galaxy S5,Bom,16GB,Dourado,59,0,1,0,1


In [23]:
productos_visitados.count()

fecha           52146
person          52146
sku             52146
model           52146
condition       52146
storage         52146
color           52146
cant_eventos    52146
compras_prod    52146
vistas_prod     52146
otros           52146
label           52146
dtype: int64

In [24]:
print('1s ', productos_visitados[productos_visitados['label'] == 1].shape)
print('0s ', productos_visitados[productos_visitados['label'] == 0].shape)

1s  (13707, 12)
0s  (38439, 12)


In [25]:
otros = sets.loc[:, cols]
otros = otros[(otros['event'] == 'brand listing') | (otros['event'] == 'generic listing') | (otros['event'] == 'lead') | (otros['event'] == 'staticpage')]
otros.drop(['event'], inplace=True, axis=1)
otros['compras_prod'] = 0
otros['vistas_prod'] = 0
otros['otros'] = 1
frecuencia_otros = otros[['person']]
frecuencia_otros['cant_otros'] = 1
frecuencia_otros = pd.DataFrame({'cant_otros': frecuencia_otros.groupby('person').aggregate(sum)['cant_otros']}).reset_index()
otros.head()

,fecha,person,sku,model,condition,storage,color,cant_eventos,label,compras_prod,vistas_prod,otros
38,2018-05-14 23:50:23,ad93850f,NaN,NaN,NaN,NaN,NaN,65,0,0,0,1
39,2018-05-14 23:50:44,ad93850f,NaN,NaN,NaN,NaN,NaN,65,0,0,0,1
40,2018-05-14 23:51:11,ad93850f,NaN,NaN,NaN,NaN,NaN,65,0,0,0,1
41,2018-05-14 23:51:42,ad93850f,NaN,NaN,NaN,NaN,NaN,65,0,0,0,1
42,2018-05-14 23:52:21,ad93850f,NaN,NaN,NaN,NaN,NaN,65,0,0,0,1


In [26]:
otros.count()

fecha           192537
person          192537
sku                  0
model              431
condition            0
storage              0
color                0
cant_eventos    192537
label           192537
compras_prod    192537
vistas_prod     192537
otros           192537
dtype: int64

In [27]:
frecuencia_otros.head()

,person,cant_otros
0,0008ed71,1
1,000c79fe,1
2,001802e4,4
3,0019e639,193
4,001b0bf9,2


In [28]:
print('1s ', otros[otros['label'] == 1].shape)
print('0s ', otros[otros['label'] == 0].shape)

1s  (19452, 12)
0s  (173085, 12)


In [29]:
# otros = otros.drop_duplicates()
# otros.count()

In [30]:
# Eliminamos de otros las personas que estan en productos comprados
otros = otros.merge(productos_comprados, on='person', how='left')
otros = otros[otros['fecha_y'].isnull()]
otros = otros.loc[:,cols_x]
otros.rename(columns=rename, inplace=True)
otros.head()

,fecha,person,sku,model,condition,storage,color,cant_eventos,compras_prod,vistas_prod,otros,label
3052,2018-05-17 17:16:36,9bb3af27,NaN,NaN,NaN,NaN,NaN,59,0,0,1,1
3053,2018-05-17 17:24:56,9bb3af27,NaN,NaN,NaN,NaN,NaN,59,0,0,1,1
3054,2018-05-18 00:11:31,9bb3af27,NaN,NaN,NaN,NaN,NaN,59,0,0,1,1
3055,2018-05-18 00:12:18,9bb3af27,NaN,NaN,NaN,NaN,NaN,59,0,0,1,1
3056,2018-05-18 00:12:38,9bb3af27,NaN,NaN,NaN,NaN,NaN,59,0,0,1,1


In [31]:
otros.count()

fecha           17348
person          17348
sku                 0
model              32
condition           0
storage             0
color               0
cant_eventos    17348
compras_prod    17348
vistas_prod     17348
otros           17348
label           17348
dtype: int64

In [32]:
print('1s ', otros[otros['label'] == 1].shape)
print('0s ', otros[otros['label'] == 0].shape)

1s  (4867, 12)
0s  (12481, 12)


In [33]:
# Eliminamos de otros las personas que estan en productos visitados
otros = otros.merge(productos_visitados, on='person', how='left')
otros = otros[otros['fecha_y'].isnull()]
otros = otros.loc[:,cols_x]
otros.rename(columns=rename, inplace=True)
otros.head()

,fecha,person,sku,model,condition,storage,color,cant_eventos,compras_prod,vistas_prod,otros,label
60635,2018-05-29 20:56:57,7a3184e8,NaN,NaN,NaN,NaN,NaN,3,0,0,1,0
60636,2018-05-29 19:45:16,f33e2cc5,NaN,NaN,NaN,NaN,NaN,4,0,0,1,0
61533,2018-05-19 10:31:43,fb3c6e61,NaN,NaN,NaN,NaN,NaN,4,0,0,1,0
145036,2018-05-29 21:21:02,4042a213,NaN,NaN,NaN,NaN,NaN,6,0,0,1,0
145267,2018-05-18 14:20:44,9a84d509,NaN,NaN,NaN,NaN,NaN,12,0,0,1,0


In [34]:
otros.count()

fecha           568
person          568
sku               0
model             2
condition         0
storage           0
color             0
cant_eventos    568
compras_prod    568
vistas_prod     568
otros           568
label           568
dtype: int64

In [35]:
print('1s ', otros[otros['label'] == 1].shape)
print('0s ', otros[otros['label'] == 0].shape)

1s  (46, 12)
0s  (522, 12)


In [36]:
#Caracterisitcas del dispositivos con el que el usuario visito el sitio
visitas = sets.loc[:,cols_visit]
features_usuario = visitas.loc[visitas.event == 'visited site']
features_usuario.drop(['event'], inplace=True, axis=1)
features_usuario['visitas_sitio'] = 1
features_usuario.head()

,fecha,person,new_vs_returning,region,country,device_type,screen_resolution,operating_system_version,browser_version,cant_eventos,label,visitas_sitio
60,2018-05-14 23:50:22,ad93850f,New,Sao Paulo,Brazil,Smartphone,360x640,Android 5.1.1,Chrome Mobile 66.0,65,0,1
61,2018-05-16 02:48:13,ad93850f,Returning,Sao Paulo,Brazil,Smartphone,360x640,Android 5.1.1,Chrome Mobile 66.0,65,0,1
62,2018-05-18 00:11:26,ad93850f,Returning,Sao Paulo,Brazil,Smartphone,360x640,Android 5.1.1,Chrome Mobile 66.0,65,0,1
63,2018-05-18 22:11:46,ad93850f,Returning,Sao Paulo,Brazil,Smartphone,360x640,Android 5.1.1,Chrome Mobile 66.0,65,0,1
64,2018-05-22 22:41:31,ad93850f,Returning,Sao Paulo,Brazil,Smartphone,360x640,Android 5.1.1,Chrome Mobile 66.0,65,0,1


In [37]:
features_usuario.count()

fecha                       102299
person                      102299
new_vs_returning            102299
region                      102299
country                     102299
device_type                 102299
screen_resolution           102297
operating_system_version    102299
browser_version             102299
cant_eventos                102299
label                       102299
visitas_sitio               102299
dtype: int64

In [38]:
print('1s ', otros[otros['label'] == 1].shape)
print('0s ', otros[otros['label'] == 0].shape)

1s  (46, 12)
0s  (522, 12)


In [39]:
# features_usuario = features_usuario.drop_duplicates()
# features_usuario.count()

In [40]:
del productos
del visitas

In [41]:
user_prod_comprados = productos_comprados.merge(features_usuario, on="person", how="left")
user_prod_comprados.head()

,fecha_x,person,sku,model,condition,storage,color,cant_eventos_x,label_x,compras_prod,...,new_vs_returning,region,country,device_type,screen_resolution,operating_system_version,browser_version,cant_eventos_y,label_y,visitas_sitio
0,2018-05-14 23:54:19,ad93850f,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,65,0,1,...,New,Sao Paulo,Brazil,Smartphone,360x640,Android 5.1.1,Chrome Mobile 66.0,65.0,0.0,1.0
1,2018-05-14 23:54:19,ad93850f,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,65,0,1,...,Returning,Sao Paulo,Brazil,Smartphone,360x640,Android 5.1.1,Chrome Mobile 66.0,65.0,0.0,1.0
2,2018-05-14 23:54:19,ad93850f,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,65,0,1,...,Returning,Sao Paulo,Brazil,Smartphone,360x640,Android 5.1.1,Chrome Mobile 66.0,65.0,0.0,1.0
3,2018-05-14 23:54:19,ad93850f,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,65,0,1,...,Returning,Sao Paulo,Brazil,Smartphone,360x640,Android 5.1.1,Chrome Mobile 66.0,65.0,0.0,1.0
4,2018-05-14 23:54:19,ad93850f,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,65,0,1,...,Returning,Sao Paulo,Brazil,Smartphone,360x640,Android 5.1.1,Chrome Mobile 66.0,65.0,0.0,1.0


In [42]:
user_prod_comprados.count()

fecha_x                     487507
person                      487507
sku                         487507
model                       487507
condition                   487507
storage                     487507
color                       487507
cant_eventos_x              487507
label_x                     487507
compras_prod                487507
vistas_prod                 487507
otros                       487507
fecha_y                     487040
new_vs_returning            487040
region                      487040
country                     487040
device_type                 487040
screen_resolution           487027
operating_system_version    487040
browser_version             487040
cant_eventos_y              487040
label_y                     487040
visitas_sitio               487040
dtype: int64

In [43]:
print('1s_x ', user_prod_comprados[user_prod_comprados['label_x'] == 1].shape)
print('0s_x ', user_prod_comprados[user_prod_comprados['label_x'] == 0].shape)
print('1s_y ', user_prod_comprados[user_prod_comprados['label_y'] == 1].shape)
print('0s_y ', user_prod_comprados[user_prod_comprados['label_y'] == 0].shape)
print('1s_x y 1s_y', user_prod_comprados[(user_prod_comprados['label_x'] == 1) & (user_prod_comprados['label_y'] == 1)].shape)
print('0s_x y 0s_y', user_prod_comprados[(user_prod_comprados['label_x'] == 0) & (user_prod_comprados['label_y'] == 0)].shape)
print('1s_x y 0s_y', user_prod_comprados[(user_prod_comprados['label_x'] == 1) & (user_prod_comprados['label_y'] == 0)].shape)
print('0s_x y 1s_y ', user_prod_comprados[(user_prod_comprados['label_x'] == 0) & (user_prod_comprados['label_y'] == 1)].shape)

1s_x  (64127, 23)
0s_x  (423380, 23)
1s_y  (64119, 23)
0s_y  (422921, 23)
1s_x y 1s_y (64119, 23)
0s_x y 0s_y (422921, 23)
1s_x y 0s_y (0, 23)
0s_x y 1s_y  (0, 23)


In [44]:
user_prod_vistas = productos_visitados.merge(features_usuario, on="person", how="left")
user_prod_vistas.head()

,fecha_x,person,sku,model,condition,storage,color,cant_eventos_x,compras_prod,vistas_prod,...,new_vs_returning,region,country,device_type,screen_resolution,operating_system_version,browser_version,cant_eventos_y,label_y,visitas_sitio
0,2018-05-18 00:18:24,9bb3af27,3371.0,Samsung Galaxy S6 Flat,Bom,32GB,Dourado,59,0,1,...,New,Minas Gerais,Brazil,Smartphone,320x534,Android 6,Chrome Mobile 66.0,59.0,1.0,1.0
1,2018-05-18 00:18:24,9bb3af27,3371.0,Samsung Galaxy S6 Flat,Bom,32GB,Dourado,59,0,1,...,Returning,Minas Gerais,Brazil,Smartphone,320x534,Android 6,Chrome Mobile 66.0,59.0,1.0,1.0
2,2018-05-18 00:18:24,9bb3af27,3371.0,Samsung Galaxy S6 Flat,Bom,32GB,Dourado,59,0,1,...,Returning,Minas Gerais,Brazil,Smartphone,320x534,Android 6,Chrome Mobile 66.0,59.0,1.0,1.0
3,2018-05-18 00:18:24,9bb3af27,3371.0,Samsung Galaxy S6 Flat,Bom,32GB,Dourado,59,0,1,...,Returning,Minas Gerais,Brazil,Smartphone,320x534,Android 6,Chrome Mobile 66.0,59.0,1.0,1.0
4,2018-05-18 00:15:33,9bb3af27,2741.0,Samsung Galaxy S5,Bom,16GB,Preto,59,0,1,...,New,Minas Gerais,Brazil,Smartphone,320x534,Android 6,Chrome Mobile 66.0,59.0,1.0,1.0


In [45]:
user_prod_vistas.count()

fecha_x                     582196
person                      582196
sku                         582196
model                       582196
condition                   582196
storage                     582196
color                       582196
cant_eventos_x              582196
compras_prod                582196
vistas_prod                 582196
otros                       582196
label_x                     582196
fecha_y                     582073
new_vs_returning            582073
region                      582073
country                     582073
device_type                 582073
screen_resolution           582073
operating_system_version    582073
browser_version             582073
cant_eventos_y              582073
label_y                     582073
visitas_sitio               582073
dtype: int64

In [46]:
print('1s_x ', user_prod_vistas[user_prod_vistas['label_x'] == 1].shape)
print('0s_x ', user_prod_vistas[user_prod_vistas['label_x'] == 0].shape)
print('1s_y ', user_prod_vistas[user_prod_vistas['label_y'] == 1].shape)
print('0s_y ', user_prod_vistas[user_prod_vistas['label_y'] == 0].shape)
print('1s_x y 1s_y', user_prod_vistas[(user_prod_vistas['label_x'] == 1) & (user_prod_vistas['label_y'] == 1)].shape)
print('0s_x y 0s_y', user_prod_vistas[(user_prod_vistas['label_x'] == 0) & (user_prod_vistas['label_y'] == 0)].shape)
print('1s_x y 0s_y', user_prod_vistas[(user_prod_vistas['label_x'] == 1) & (user_prod_vistas['label_y'] == 0)].shape)
print('0s_x y 1s_y ', user_prod_vistas[(user_prod_vistas['label_x'] == 0) & (user_prod_vistas['label_y'] == 1)].shape)

1s_x  (204354, 23)
0s_x  (377842, 23)
1s_y  (204244, 23)
0s_y  (377829, 23)
1s_x y 1s_y (204244, 23)
0s_x y 0s_y (377829, 23)
1s_x y 0s_y (0, 23)
0s_x y 1s_y  (0, 23)


In [47]:
otros = otros.merge(features_usuario, on="person", how="outer")
otros.head()

,fecha_x,person,sku,model,condition,storage,color,cant_eventos_x,compras_prod,vistas_prod,...,new_vs_returning,region,country,device_type,screen_resolution,operating_system_version,browser_version,cant_eventos_y,label_y,visitas_sitio
0,2018-05-29 20:56:57,7a3184e8,NaN,NaN,NaN,NaN,NaN,3.0,0.0,0.0,...,New,Espirito Santo,Brazil,Smartphone,360x640,Android 6,Chrome Mobile 66.0,3.0,0.0,1.0
1,2018-05-29 19:45:16,f33e2cc5,NaN,NaN,NaN,NaN,NaN,4.0,0.0,0.0,...,New,Sao Paulo,Brazil,Smartphone,360x640,Android 7,Chrome Mobile 66.0,4.0,0.0,1.0
2,2018-05-19 10:31:43,fb3c6e61,NaN,NaN,NaN,NaN,NaN,4.0,0.0,0.0,...,New,Pernambuco,Brazil,Smartphone,360x640,Android 6,Chrome Mobile 66.0,4.0,0.0,1.0
3,2018-05-29 21:21:02,4042a213,NaN,NaN,NaN,NaN,NaN,6.0,0.0,0.0,...,New,Rio de Janeiro,Brazil,Smartphone,320x570,Android 6,Chrome Mobile 64.0,6.0,0.0,1.0
4,2018-05-18 14:20:44,9a84d509,NaN,NaN,NaN,NaN,NaN,12.0,0.0,0.0,...,Returning,Ceara,Brazil,Smartphone,360x640,Android 7,Chrome Mobile 66.0,12.0,0.0,1.0


In [48]:
otros.count()

fecha_x                       3309
person                      105229
sku                              0
model                            2
condition                        0
storage                          0
color                            0
cant_eventos_x                3309
compras_prod                  3309
vistas_prod                   3309
otros                         3309
label_x                       3309
fecha_y                     105221
new_vs_returning            105221
region                      105221
country                     105221
device_type                 105221
screen_resolution           105219
operating_system_version    105221
browser_version             105221
cant_eventos_y              105221
label_y                     105221
visitas_sitio               105221
dtype: int64

In [49]:
print('1s_x ', otros[otros['label_x'] == 1].shape)
print('0s_x ', otros[otros['label_x'] == 0].shape)
print('1s_y ', otros[otros['label_y'] == 1].shape)
print('0s_y ', otros[otros['label_y'] == 0].shape)
print('1s_x y 1s_y', otros[(otros['label_x'] == 1) & (otros['label_y'] == 1)].shape)
print('0s_x y 0s_y', otros[(otros['label_x'] == 0) & (otros['label_y'] == 0)].shape)
print('1s_x y 0s_y', otros[(otros['label_x'] == 1) & (otros['label_y'] == 0)].shape)
print('0s_x y 1s_y ', otros[(otros['label_x'] == 0) & (otros['label_y'] == 1)].shape)

1s_x  (63, 23)
0s_x  (3246, 23)
1s_y  (8956, 23)
0s_y  (96265, 23)
1s_x y 1s_y (63, 23)
0s_x y 0s_y (3238, 23)
1s_x y 0s_y (0, 23)
0s_x y 1s_y  (0, 23)


In [50]:
del features_usuario
del productos_comprados
del productos_visitados

In [51]:
#concateno
frames = [user_prod_comprados, user_prod_vistas, otros]
sets = pd.concat(frames)
sets.head()

,browser_version,cant_eventos_x,cant_eventos_y,color,compras_prod,condition,country,device_type,fecha_x,fecha_y,...,new_vs_returning,operating_system_version,otros,person,region,screen_resolution,sku,storage,visitas_sitio,vistas_prod
0,Chrome Mobile 66.0,65.0,65.0,Cinza espacial,1.0,Muito Bom,Brazil,Smartphone,2018-05-14 23:54:19,2018-05-14 23:50:22,...,New,Android 5.1.1,0.0,ad93850f,Sao Paulo,360x640,304.0,32GB,1.0,0.0
1,Chrome Mobile 66.0,65.0,65.0,Cinza espacial,1.0,Muito Bom,Brazil,Smartphone,2018-05-14 23:54:19,2018-05-16 02:48:13,...,Returning,Android 5.1.1,0.0,ad93850f,Sao Paulo,360x640,304.0,32GB,1.0,0.0
2,Chrome Mobile 66.0,65.0,65.0,Cinza espacial,1.0,Muito Bom,Brazil,Smartphone,2018-05-14 23:54:19,2018-05-18 00:11:26,...,Returning,Android 5.1.1,0.0,ad93850f,Sao Paulo,360x640,304.0,32GB,1.0,0.0
3,Chrome Mobile 66.0,65.0,65.0,Cinza espacial,1.0,Muito Bom,Brazil,Smartphone,2018-05-14 23:54:19,2018-05-18 22:11:46,...,Returning,Android 5.1.1,0.0,ad93850f,Sao Paulo,360x640,304.0,32GB,1.0,0.0
4,Chrome Mobile 66.0,65.0,65.0,Cinza espacial,1.0,Muito Bom,Brazil,Smartphone,2018-05-14 23:54:19,2018-05-22 22:41:31,...,Returning,Android 5.1.1,0.0,ad93850f,Sao Paulo,360x640,304.0,32GB,1.0,0.0


In [52]:
del frames
del user_prod_comprados
del user_prod_vistas
del otros

In [53]:
sets.count()

browser_version             1174334
cant_eventos_x              1073012
cant_eventos_y              1174334
color                       1069703
compras_prod                1073012
condition                   1069703
country                     1174334
device_type                 1174334
fecha_x                     1073012
fecha_y                     1174334
label_x                     1073012
label_y                     1174334
model                       1069705
new_vs_returning            1174334
operating_system_version    1174334
otros                       1073012
person                      1174932
region                      1174334
screen_resolution           1174319
sku                         1069703
storage                     1069703
visitas_sitio               1174334
vistas_prod                 1073012
dtype: int64

In [54]:
# Agregamos las frecuencias de cada persona de compras, vistas y otros
sets = sets.merge(frecuencia_compras, on = 'person', how = 'left')
sets = sets.merge(frecuencia_vistas, on = 'person', how = 'left')
sets = sets.merge(frecuencia_otros, on = 'person', how = 'left')
sets.head()

,browser_version,cant_eventos_x,cant_eventos_y,color,compras_prod,condition,country,device_type,fecha_x,fecha_y,...,person,region,screen_resolution,sku,storage,visitas_sitio,vistas_prod,cant_compras,cant_vistas,cant_otros
0,Chrome Mobile 66.0,65.0,65.0,Cinza espacial,1.0,Muito Bom,Brazil,Smartphone,2018-05-14 23:54:19,2018-05-14 23:50:22,...,ad93850f,Sao Paulo,360x640,304.0,32GB,1.0,0.0,1.0,20.0,22.0
1,Chrome Mobile 66.0,65.0,65.0,Cinza espacial,1.0,Muito Bom,Brazil,Smartphone,2018-05-14 23:54:19,2018-05-16 02:48:13,...,ad93850f,Sao Paulo,360x640,304.0,32GB,1.0,0.0,1.0,20.0,22.0
2,Chrome Mobile 66.0,65.0,65.0,Cinza espacial,1.0,Muito Bom,Brazil,Smartphone,2018-05-14 23:54:19,2018-05-18 00:11:26,...,ad93850f,Sao Paulo,360x640,304.0,32GB,1.0,0.0,1.0,20.0,22.0
3,Chrome Mobile 66.0,65.0,65.0,Cinza espacial,1.0,Muito Bom,Brazil,Smartphone,2018-05-14 23:54:19,2018-05-18 22:11:46,...,ad93850f,Sao Paulo,360x640,304.0,32GB,1.0,0.0,1.0,20.0,22.0
4,Chrome Mobile 66.0,65.0,65.0,Cinza espacial,1.0,Muito Bom,Brazil,Smartphone,2018-05-14 23:54:19,2018-05-22 22:41:31,...,ad93850f,Sao Paulo,360x640,304.0,32GB,1.0,0.0,1.0,20.0,22.0


In [55]:
sets['cant_compras'] = sets['cant_compras'].fillna(value=0) 
sets['cant_vistas'] = sets['cant_vistas'].fillna(value=0) 
sets['cant_otros'] = sets['cant_otros'].fillna(value=0) 
del frecuencia_compras
del frecuencia_vistas
del frecuencia_otros

In [56]:
# sets = sets.drop_duplicates()

In [57]:
# Rellenamos compras_prod  vistas_prod  otros y visitas_sitio con 0
sets['compras_prod'] = sets['compras_prod'].fillna(value=0) 
sets['vistas_prod'] = sets['vistas_prod'].fillna(value=0) 
sets['otros'] = sets['otros'].fillna(value=0) 
sets['visitas_sitio'] = sets['visitas_sitio'].fillna(value=0) 

In [58]:
sets.count()

browser_version             1174334
cant_eventos_x              1073012
cant_eventos_y              1174334
color                       1069703
compras_prod                1174932
condition                   1069703
country                     1174334
device_type                 1174334
fecha_x                     1073012
fecha_y                     1174334
label_x                     1073012
label_y                     1174334
model                       1069705
new_vs_returning            1174334
operating_system_version    1174334
otros                       1174932
person                      1174932
region                      1174334
screen_resolution           1174319
sku                         1069703
storage                     1069703
visitas_sitio               1174932
vistas_prod                 1174932
cant_compras                1174932
cant_vistas                 1174932
cant_otros                  1174932
dtype: int64

In [59]:
# Rellenamos label_x y label_y con 0 y nos quedamos con label
sets['label_x'] = sets['label_x'].fillna(value=0) 
sets['label_y'] = sets['label_y'].fillna(value=0) 
sets.loc[(sets['label_x'] == 1) | (sets['label_y'] == 1), 'label'] = 1
sets['label'] = sets['label'].fillna(value=0) 
sets.drop('label_x',axis=1, inplace=True)
sets.drop('label_y',axis=1, inplace=True)
sets.head()

,browser_version,cant_eventos_x,cant_eventos_y,color,compras_prod,condition,country,device_type,fecha_x,fecha_y,...,region,screen_resolution,sku,storage,visitas_sitio,vistas_prod,cant_compras,cant_vistas,cant_otros,label
0,Chrome Mobile 66.0,65.0,65.0,Cinza espacial,1.0,Muito Bom,Brazil,Smartphone,2018-05-14 23:54:19,2018-05-14 23:50:22,...,Sao Paulo,360x640,304.0,32GB,1.0,0.0,1.0,20.0,22.0,0.0
1,Chrome Mobile 66.0,65.0,65.0,Cinza espacial,1.0,Muito Bom,Brazil,Smartphone,2018-05-14 23:54:19,2018-05-16 02:48:13,...,Sao Paulo,360x640,304.0,32GB,1.0,0.0,1.0,20.0,22.0,0.0
2,Chrome Mobile 66.0,65.0,65.0,Cinza espacial,1.0,Muito Bom,Brazil,Smartphone,2018-05-14 23:54:19,2018-05-18 00:11:26,...,Sao Paulo,360x640,304.0,32GB,1.0,0.0,1.0,20.0,22.0,0.0
3,Chrome Mobile 66.0,65.0,65.0,Cinza espacial,1.0,Muito Bom,Brazil,Smartphone,2018-05-14 23:54:19,2018-05-18 22:11:46,...,Sao Paulo,360x640,304.0,32GB,1.0,0.0,1.0,20.0,22.0,0.0
4,Chrome Mobile 66.0,65.0,65.0,Cinza espacial,1.0,Muito Bom,Brazil,Smartphone,2018-05-14 23:54:19,2018-05-22 22:41:31,...,Sao Paulo,360x640,304.0,32GB,1.0,0.0,1.0,20.0,22.0,0.0


In [60]:
print('1s ', sets[sets['label'] == 1].shape)
print('0s ',sets[sets['label'] == 0].shape)

1s  (277437, 25)
0s  (897495, 25)


In [61]:
#  Nos quedamos con cant_eventos_x (ya que son iguales con cant_eventos_y) convertimos a int y Rellenamos con 0
sets['cant_eventos'] = sets['cant_eventos_x']
sets['cant_eventos'] = sets['cant_eventos'].fillna(value=0) 
sets['cant_eventos'] = sets['cant_eventos'].astype('int')
sets.drop('cant_eventos_x',axis=1, inplace=True)
sets.drop('cant_eventos_y',axis=1, inplace=True)
sets.head()

,browser_version,color,compras_prod,condition,country,device_type,fecha_x,fecha_y,model,new_vs_returning,...,screen_resolution,sku,storage,visitas_sitio,vistas_prod,cant_compras,cant_vistas,cant_otros,label,cant_eventos
0,Chrome Mobile 66.0,Cinza espacial,1.0,Muito Bom,Brazil,Smartphone,2018-05-14 23:54:19,2018-05-14 23:50:22,iPhone 5s,New,...,360x640,304.0,32GB,1.0,0.0,1.0,20.0,22.0,0.0,65
1,Chrome Mobile 66.0,Cinza espacial,1.0,Muito Bom,Brazil,Smartphone,2018-05-14 23:54:19,2018-05-16 02:48:13,iPhone 5s,Returning,...,360x640,304.0,32GB,1.0,0.0,1.0,20.0,22.0,0.0,65
2,Chrome Mobile 66.0,Cinza espacial,1.0,Muito Bom,Brazil,Smartphone,2018-05-14 23:54:19,2018-05-18 00:11:26,iPhone 5s,Returning,...,360x640,304.0,32GB,1.0,0.0,1.0,20.0,22.0,0.0,65
3,Chrome Mobile 66.0,Cinza espacial,1.0,Muito Bom,Brazil,Smartphone,2018-05-14 23:54:19,2018-05-18 22:11:46,iPhone 5s,Returning,...,360x640,304.0,32GB,1.0,0.0,1.0,20.0,22.0,0.0,65
4,Chrome Mobile 66.0,Cinza espacial,1.0,Muito Bom,Brazil,Smartphone,2018-05-14 23:54:19,2018-05-22 22:41:31,iPhone 5s,Returning,...,360x640,304.0,32GB,1.0,0.0,1.0,20.0,22.0,0.0,65


In [62]:
# Rellenamos model con 0 para luego hacer un split entre marca y modelo
sets['model'] = sets['model'].fillna(value='0') 

In [63]:
# Rellenamos los demas con ''
sets = sets.fillna(value='') 

In [64]:
sets.count()

browser_version             1174932
color                       1174932
compras_prod                1174932
condition                   1174932
country                     1174932
device_type                 1174932
fecha_x                     1174932
fecha_y                     1174932
model                       1174932
new_vs_returning            1174932
operating_system_version    1174932
otros                       1174932
person                      1174932
region                      1174932
screen_resolution           1174932
sku                         1174932
storage                     1174932
visitas_sitio               1174932
vistas_prod                 1174932
cant_compras                1174932
cant_vistas                 1174932
cant_otros                  1174932
label                       1174932
cant_eventos                1174932
dtype: int64

In [65]:
sets.person.describe()

count      1174932
unique       19414
top       ecd79d0a
freq         33950
Name: person, dtype: object

## Categorizacion de los features

In [66]:
# Rellenamos cantidad de eventos y convertimos a int
sets['cant_eventos'] = sets['cant_eventos'].fillna(value=0) 
sets['cant_eventos'] = sets['cant_eventos'].astype('int')

sets['cant_compras'] = sets['cant_compras'].fillna(value=0) 
sets['cant_compras'] = sets['cant_compras'].astype('int')
sets['cant_vistas'] = sets['cant_vistas'].fillna(value=0) 
sets['cant_vistas'] = sets['cant_vistas'].astype('int')
sets['cant_otros'] = sets['cant_otros'].fillna(value=0) 
sets['cant_otros'] = sets['cant_otros'].astype('int')

In [67]:
# Partimos el feature model en marca y modelo
sets['model'] = sets['model'].apply(lambda x: x.split(' '))
sets['marca'] = sets['model'].apply(lambda x: "apple" if str(x[0]) == "iPhone" else str(x[0]))
sets['modelo'] = sets['model'].apply(lambda x: ' '.join(x[1:]) if len(x)>1 else "0" )
sets.drop('model', axis=1, inplace=True)
sets.head()

,browser_version,color,compras_prod,condition,country,device_type,fecha_x,fecha_y,new_vs_returning,operating_system_version,...,storage,visitas_sitio,vistas_prod,cant_compras,cant_vistas,cant_otros,label,cant_eventos,marca,modelo
0,Chrome Mobile 66.0,Cinza espacial,1.0,Muito Bom,Brazil,Smartphone,2018-05-14 23:54:19,2018-05-14 23:50:22,New,Android 5.1.1,...,32GB,1.0,0.0,1,20,22,0.0,65,apple,5s
1,Chrome Mobile 66.0,Cinza espacial,1.0,Muito Bom,Brazil,Smartphone,2018-05-14 23:54:19,2018-05-16 02:48:13,Returning,Android 5.1.1,...,32GB,1.0,0.0,1,20,22,0.0,65,apple,5s
2,Chrome Mobile 66.0,Cinza espacial,1.0,Muito Bom,Brazil,Smartphone,2018-05-14 23:54:19,2018-05-18 00:11:26,Returning,Android 5.1.1,...,32GB,1.0,0.0,1,20,22,0.0,65,apple,5s
3,Chrome Mobile 66.0,Cinza espacial,1.0,Muito Bom,Brazil,Smartphone,2018-05-14 23:54:19,2018-05-18 22:11:46,Returning,Android 5.1.1,...,32GB,1.0,0.0,1,20,22,0.0,65,apple,5s
4,Chrome Mobile 66.0,Cinza espacial,1.0,Muito Bom,Brazil,Smartphone,2018-05-14 23:54:19,2018-05-22 22:41:31,Returning,Android 5.1.1,...,32GB,1.0,0.0,1,20,22,0.0,65,apple,5s


In [68]:
# creo una nueva columna person_int para entrenar transformando el valor alfanumerico de person a numerico
sets['person_int'] = labelencoder.fit_transform(sets['person'])
sets.head()

,browser_version,color,compras_prod,condition,country,device_type,fecha_x,fecha_y,new_vs_returning,operating_system_version,...,visitas_sitio,vistas_prod,cant_compras,cant_vistas,cant_otros,label,cant_eventos,marca,modelo,person_int
0,Chrome Mobile 66.0,Cinza espacial,1.0,Muito Bom,Brazil,Smartphone,2018-05-14 23:54:19,2018-05-14 23:50:22,New,Android 5.1.1,...,1.0,0.0,1,20,22,0.0,65,apple,5s,13068
1,Chrome Mobile 66.0,Cinza espacial,1.0,Muito Bom,Brazil,Smartphone,2018-05-14 23:54:19,2018-05-16 02:48:13,Returning,Android 5.1.1,...,1.0,0.0,1,20,22,0.0,65,apple,5s,13068
2,Chrome Mobile 66.0,Cinza espacial,1.0,Muito Bom,Brazil,Smartphone,2018-05-14 23:54:19,2018-05-18 00:11:26,Returning,Android 5.1.1,...,1.0,0.0,1,20,22,0.0,65,apple,5s,13068
3,Chrome Mobile 66.0,Cinza espacial,1.0,Muito Bom,Brazil,Smartphone,2018-05-14 23:54:19,2018-05-18 22:11:46,Returning,Android 5.1.1,...,1.0,0.0,1,20,22,0.0,65,apple,5s,13068
4,Chrome Mobile 66.0,Cinza espacial,1.0,Muito Bom,Brazil,Smartphone,2018-05-14 23:54:19,2018-05-22 22:41:31,Returning,Android 5.1.1,...,1.0,0.0,1,20,22,0.0,65,apple,5s,13068


In [69]:
# convertimos a integer el valor flotante de compras_prod
sets['compras_prod'] = sets['compras_prod'].astype('int')
sets.head()

,browser_version,color,compras_prod,condition,country,device_type,fecha_x,fecha_y,new_vs_returning,operating_system_version,...,visitas_sitio,vistas_prod,cant_compras,cant_vistas,cant_otros,label,cant_eventos,marca,modelo,person_int
0,Chrome Mobile 66.0,Cinza espacial,1,Muito Bom,Brazil,Smartphone,2018-05-14 23:54:19,2018-05-14 23:50:22,New,Android 5.1.1,...,1.0,0.0,1,20,22,0.0,65,apple,5s,13068
1,Chrome Mobile 66.0,Cinza espacial,1,Muito Bom,Brazil,Smartphone,2018-05-14 23:54:19,2018-05-16 02:48:13,Returning,Android 5.1.1,...,1.0,0.0,1,20,22,0.0,65,apple,5s,13068
2,Chrome Mobile 66.0,Cinza espacial,1,Muito Bom,Brazil,Smartphone,2018-05-14 23:54:19,2018-05-18 00:11:26,Returning,Android 5.1.1,...,1.0,0.0,1,20,22,0.0,65,apple,5s,13068
3,Chrome Mobile 66.0,Cinza espacial,1,Muito Bom,Brazil,Smartphone,2018-05-14 23:54:19,2018-05-18 22:11:46,Returning,Android 5.1.1,...,1.0,0.0,1,20,22,0.0,65,apple,5s,13068
4,Chrome Mobile 66.0,Cinza espacial,1,Muito Bom,Brazil,Smartphone,2018-05-14 23:54:19,2018-05-22 22:41:31,Returning,Android 5.1.1,...,1.0,0.0,1,20,22,0.0,65,apple,5s,13068


In [70]:
# convertimos a integer el valor flotante de vistas_prod
sets['vistas_prod'] = sets['vistas_prod'].astype('int')
sets.head()

,browser_version,color,compras_prod,condition,country,device_type,fecha_x,fecha_y,new_vs_returning,operating_system_version,...,visitas_sitio,vistas_prod,cant_compras,cant_vistas,cant_otros,label,cant_eventos,marca,modelo,person_int
0,Chrome Mobile 66.0,Cinza espacial,1,Muito Bom,Brazil,Smartphone,2018-05-14 23:54:19,2018-05-14 23:50:22,New,Android 5.1.1,...,1.0,0,1,20,22,0.0,65,apple,5s,13068
1,Chrome Mobile 66.0,Cinza espacial,1,Muito Bom,Brazil,Smartphone,2018-05-14 23:54:19,2018-05-16 02:48:13,Returning,Android 5.1.1,...,1.0,0,1,20,22,0.0,65,apple,5s,13068
2,Chrome Mobile 66.0,Cinza espacial,1,Muito Bom,Brazil,Smartphone,2018-05-14 23:54:19,2018-05-18 00:11:26,Returning,Android 5.1.1,...,1.0,0,1,20,22,0.0,65,apple,5s,13068
3,Chrome Mobile 66.0,Cinza espacial,1,Muito Bom,Brazil,Smartphone,2018-05-14 23:54:19,2018-05-18 22:11:46,Returning,Android 5.1.1,...,1.0,0,1,20,22,0.0,65,apple,5s,13068
4,Chrome Mobile 66.0,Cinza espacial,1,Muito Bom,Brazil,Smartphone,2018-05-14 23:54:19,2018-05-22 22:41:31,Returning,Android 5.1.1,...,1.0,0,1,20,22,0.0,65,apple,5s,13068


In [71]:
# convertimos a integer el valor flotante de otros
sets['otros'] = sets['otros'].astype('int')
sets.head()

,browser_version,color,compras_prod,condition,country,device_type,fecha_x,fecha_y,new_vs_returning,operating_system_version,...,visitas_sitio,vistas_prod,cant_compras,cant_vistas,cant_otros,label,cant_eventos,marca,modelo,person_int
0,Chrome Mobile 66.0,Cinza espacial,1,Muito Bom,Brazil,Smartphone,2018-05-14 23:54:19,2018-05-14 23:50:22,New,Android 5.1.1,...,1.0,0,1,20,22,0.0,65,apple,5s,13068
1,Chrome Mobile 66.0,Cinza espacial,1,Muito Bom,Brazil,Smartphone,2018-05-14 23:54:19,2018-05-16 02:48:13,Returning,Android 5.1.1,...,1.0,0,1,20,22,0.0,65,apple,5s,13068
2,Chrome Mobile 66.0,Cinza espacial,1,Muito Bom,Brazil,Smartphone,2018-05-14 23:54:19,2018-05-18 00:11:26,Returning,Android 5.1.1,...,1.0,0,1,20,22,0.0,65,apple,5s,13068
3,Chrome Mobile 66.0,Cinza espacial,1,Muito Bom,Brazil,Smartphone,2018-05-14 23:54:19,2018-05-18 22:11:46,Returning,Android 5.1.1,...,1.0,0,1,20,22,0.0,65,apple,5s,13068
4,Chrome Mobile 66.0,Cinza espacial,1,Muito Bom,Brazil,Smartphone,2018-05-14 23:54:19,2018-05-22 22:41:31,Returning,Android 5.1.1,...,1.0,0,1,20,22,0.0,65,apple,5s,13068


In [72]:
# convertimos a integer el valor flotante de visitas_sitio
sets['visitas_sitio'] = sets['visitas_sitio'].astype('int')
sets.head()

,browser_version,color,compras_prod,condition,country,device_type,fecha_x,fecha_y,new_vs_returning,operating_system_version,...,visitas_sitio,vistas_prod,cant_compras,cant_vistas,cant_otros,label,cant_eventos,marca,modelo,person_int
0,Chrome Mobile 66.0,Cinza espacial,1,Muito Bom,Brazil,Smartphone,2018-05-14 23:54:19,2018-05-14 23:50:22,New,Android 5.1.1,...,1,0,1,20,22,0.0,65,apple,5s,13068
1,Chrome Mobile 66.0,Cinza espacial,1,Muito Bom,Brazil,Smartphone,2018-05-14 23:54:19,2018-05-16 02:48:13,Returning,Android 5.1.1,...,1,0,1,20,22,0.0,65,apple,5s,13068
2,Chrome Mobile 66.0,Cinza espacial,1,Muito Bom,Brazil,Smartphone,2018-05-14 23:54:19,2018-05-18 00:11:26,Returning,Android 5.1.1,...,1,0,1,20,22,0.0,65,apple,5s,13068
3,Chrome Mobile 66.0,Cinza espacial,1,Muito Bom,Brazil,Smartphone,2018-05-14 23:54:19,2018-05-18 22:11:46,Returning,Android 5.1.1,...,1,0,1,20,22,0.0,65,apple,5s,13068
4,Chrome Mobile 66.0,Cinza espacial,1,Muito Bom,Brazil,Smartphone,2018-05-14 23:54:19,2018-05-22 22:41:31,Returning,Android 5.1.1,...,1,0,1,20,22,0.0,65,apple,5s,13068


In [73]:
# convertimos a integer el valor flotante de label
sets['label'] = sets['label'].astype('int')
sets.head()

,browser_version,color,compras_prod,condition,country,device_type,fecha_x,fecha_y,new_vs_returning,operating_system_version,...,visitas_sitio,vistas_prod,cant_compras,cant_vistas,cant_otros,label,cant_eventos,marca,modelo,person_int
0,Chrome Mobile 66.0,Cinza espacial,1,Muito Bom,Brazil,Smartphone,2018-05-14 23:54:19,2018-05-14 23:50:22,New,Android 5.1.1,...,1,0,1,20,22,0,65,apple,5s,13068
1,Chrome Mobile 66.0,Cinza espacial,1,Muito Bom,Brazil,Smartphone,2018-05-14 23:54:19,2018-05-16 02:48:13,Returning,Android 5.1.1,...,1,0,1,20,22,0,65,apple,5s,13068
2,Chrome Mobile 66.0,Cinza espacial,1,Muito Bom,Brazil,Smartphone,2018-05-14 23:54:19,2018-05-18 00:11:26,Returning,Android 5.1.1,...,1,0,1,20,22,0,65,apple,5s,13068
3,Chrome Mobile 66.0,Cinza espacial,1,Muito Bom,Brazil,Smartphone,2018-05-14 23:54:19,2018-05-18 22:11:46,Returning,Android 5.1.1,...,1,0,1,20,22,0,65,apple,5s,13068
4,Chrome Mobile 66.0,Cinza espacial,1,Muito Bom,Brazil,Smartphone,2018-05-14 23:54:19,2018-05-22 22:41:31,Returning,Android 5.1.1,...,1,0,1,20,22,0,65,apple,5s,13068


In [74]:
# convertimos a string el valor flotante de sku
sets['sku'] = sets['sku'].astype('str')
sets.head()

,browser_version,color,compras_prod,condition,country,device_type,fecha_x,fecha_y,new_vs_returning,operating_system_version,...,visitas_sitio,vistas_prod,cant_compras,cant_vistas,cant_otros,label,cant_eventos,marca,modelo,person_int
0,Chrome Mobile 66.0,Cinza espacial,1,Muito Bom,Brazil,Smartphone,2018-05-14 23:54:19,2018-05-14 23:50:22,New,Android 5.1.1,...,1,0,1,20,22,0,65,apple,5s,13068
1,Chrome Mobile 66.0,Cinza espacial,1,Muito Bom,Brazil,Smartphone,2018-05-14 23:54:19,2018-05-16 02:48:13,Returning,Android 5.1.1,...,1,0,1,20,22,0,65,apple,5s,13068
2,Chrome Mobile 66.0,Cinza espacial,1,Muito Bom,Brazil,Smartphone,2018-05-14 23:54:19,2018-05-18 00:11:26,Returning,Android 5.1.1,...,1,0,1,20,22,0,65,apple,5s,13068
3,Chrome Mobile 66.0,Cinza espacial,1,Muito Bom,Brazil,Smartphone,2018-05-14 23:54:19,2018-05-18 22:11:46,Returning,Android 5.1.1,...,1,0,1,20,22,0,65,apple,5s,13068
4,Chrome Mobile 66.0,Cinza espacial,1,Muito Bom,Brazil,Smartphone,2018-05-14 23:54:19,2018-05-22 22:41:31,Returning,Android 5.1.1,...,1,0,1,20,22,0,65,apple,5s,13068


In [75]:
#Categorizamos los valores de los features con label encoder
sets['sku'] = labelencoder.fit_transform(sets['sku'])
sets['marca'] = labelencoder.fit_transform(sets['marca'])
sets['modelo'] = labelencoder.fit_transform(sets['modelo'])
sets['condition'] = labelencoder.fit_transform(sets['condition'])
sets['storage'] = labelencoder.fit_transform(sets['storage'])
sets['color'] = labelencoder.fit_transform(sets['color'])
sets['new_vs_returning'] = labelencoder.fit_transform(sets['new_vs_returning'])
sets['region'] = labelencoder.fit_transform(sets['region'])
sets['country'] = labelencoder.fit_transform(sets['country'])
sets['device_type'] = labelencoder.fit_transform(sets['device_type'])
sets['screen_resolution'] = labelencoder.fit_transform(sets['screen_resolution'])
sets['operating_system_version'] = labelencoder.fit_transform(sets['operating_system_version'])
sets['browser_version'] = labelencoder.fit_transform(sets['browser_version'])
sets.head()

,browser_version,color,compras_prod,condition,country,device_type,fecha_x,fecha_y,new_vs_returning,operating_system_version,...,visitas_sitio,vistas_prod,cant_compras,cant_vistas,cant_otros,label,cant_eventos,marca,modelo,person_int
0,103,21,1,4,5,2,2018-05-14 23:54:19,2018-05-14 23:50:22,1,19,...,1,0,1,20,22,0,65,8,12,13068
1,103,21,1,4,5,2,2018-05-14 23:54:19,2018-05-16 02:48:13,2,19,...,1,0,1,20,22,0,65,8,12,13068
2,103,21,1,4,5,2,2018-05-14 23:54:19,2018-05-18 00:11:26,2,19,...,1,0,1,20,22,0,65,8,12,13068
3,103,21,1,4,5,2,2018-05-14 23:54:19,2018-05-18 22:11:46,2,19,...,1,0,1,20,22,0,65,8,12,13068
4,103,21,1,4,5,2,2018-05-14 23:54:19,2018-05-22 22:41:31,2,19,...,1,0,1,20,22,0,65,8,12,13068


In [76]:
#Categorizamos la fecha del visto del producto en dia mes y dia de la semana  (ya que el anio es 2018)
sets['dia_view_prod'] = sets['fecha_x'].apply(lambda x: x.day if type(x) != str else 0).astype('int')
sets['mes_view_prod'] = sets['fecha_x'].apply(lambda x: x.month if type(x) != str else 0).astype('int')
sets['dia_sem_view_prod'] = sets['fecha_x'].apply(lambda x: x.dayofweek if type(x) != str else 0).astype('int')
sets = sets.drop('fecha_x',1)

In [77]:
#Categorizamos la fecha de visita del usuario en dia mes y dia de la semana (ya que el anio es 2018)
sets['dia_visit_user'] = sets['fecha_y'].apply(lambda x: x.day if type(x) != str else 0).astype('int')
sets['mes_visit_user'] = sets['fecha_y'].apply(lambda x: x.month if type(x) != str else 0).astype('int')
sets['dia_sem_visit_user'] = sets['fecha_y'].apply(lambda x: x.dayofweek if type(x) != str else 0).astype('int')
sets = sets.drop('fecha_y',1)

In [78]:
sets.head()

,browser_version,color,compras_prod,condition,country,device_type,new_vs_returning,operating_system_version,otros,person,...,cant_eventos,marca,modelo,person_int,dia_view_prod,mes_view_prod,dia_sem_view_prod,dia_visit_user,mes_visit_user,dia_sem_visit_user
0,103,21,1,4,5,2,1,19,0,ad93850f,...,65,8,12,13068,14,5,0,14,5,0
1,103,21,1,4,5,2,2,19,0,ad93850f,...,65,8,12,13068,14,5,0,16,5,2
2,103,21,1,4,5,2,2,19,0,ad93850f,...,65,8,12,13068,14,5,0,18,5,4
3,103,21,1,4,5,2,2,19,0,ad93850f,...,65,8,12,13068,14,5,0,18,5,4
4,103,21,1,4,5,2,2,19,0,ad93850f,...,65,8,12,13068,14,5,0,22,5,1


In [84]:
sets.tail()

,browser_version,color,compras_prod,condition,country,device_type,new_vs_returning,operating_system_version,otros,person,...,cant_eventos,marca,modelo,person_int,dia_view_prod,mes_view_prod,dia_sem_view_prod,dia_visit_user,mes_visit_user,dia_sem_visit_user
1174927,103,0,0,0,5,2,2,23,0,af3374ad,...,0,0,1,13206,0,0,0,15,5,1
1174928,103,0,0,0,5,2,1,23,0,023581f9,...,0,0,1,161,0,0,0,27,5,6
1174929,101,0,0,0,5,2,1,19,0,88b91c5d,...,0,0,1,10282,0,0,0,20,5,6
1174930,225,0,0,0,5,2,1,91,0,2adb3684,...,0,0,1,3200,0,0,0,21,4,5
1174931,225,0,0,0,5,2,1,91,0,39fa45b6,...,0,0,1,4361,0,0,0,22,4,6


In [85]:
sets.count()

browser_version             1174932
color                       1174932
compras_prod                1174932
condition                   1174932
country                     1174932
device_type                 1174932
new_vs_returning            1174932
operating_system_version    1174932
otros                       1174932
person                      1174932
region                      1174932
screen_resolution           1174932
sku                         1174932
storage                     1174932
visitas_sitio               1174932
vistas_prod                 1174932
cant_compras                1174932
cant_vistas                 1174932
cant_otros                  1174932
label                       1174932
cant_eventos                1174932
marca                       1174932
modelo                      1174932
person_int                  1174932
dia_view_prod               1174932
mes_view_prod               1174932
dia_sem_view_prod           1174932
dia_visit_user              

In [86]:
sets.person.describe()

count      1174932
unique       19414
top       ecd79d0a
freq         33950
Name: person, dtype: object

In [87]:
sets[sets['label'] == 1].shape

(277437, 30)

In [88]:
sets[sets['label'] == 0].shape

(897495, 30)

In [66]:
# Guardar Set

In [67]:
sets.to_csv('data/set_prediccion_19.csv', encoding='utf-8', index=False)

In [89]:
sets.to_csv('data/set_entrenamiento_19.csv', encoding='utf-8', index=False)